In [19]:
import folium
import pandas as pd
import geopy.distance
import json
from folium.plugins import LocateControl, Draw

# Load marker data from CSV files
def load_markers(file_path, marker_type):
    return pd.read_csv(file_path).assign(marker_type=marker_type)

# Load all marker types from individual CSV files
launch_with_transport = load_markers('berlin/launch_with_transport.csv', 'launch_with_transport')
launch_without_transport = load_markers('berlin/launch_without_transport.csv', 'launch_without_transport')
resting_points = load_markers('berlin/resting_points.csv', 'resting_point')

# Combine all markers into a single DataFrame
all_markers = pd.concat([launch_with_transport, launch_without_transport, resting_points], ignore_index=True)

# Create a map centered at Berlin, Germany with zoom level 11
my_map = folium.Map(location=[52.5200, 13.4050], zoom_start=11, tiles="OpenStreetMap")

# Create feature groups for three types of markers
launch_with_transport_layer = folium.FeatureGroup(name='Launch Points with Public Transport', show=True)
launch_without_transport_layer = folium.FeatureGroup(name='Launch Points without Public Transport', show=True)
resting_point_layer = folium.FeatureGroup(name='Resting Points', show=True)

# Function to determine marker color based on accessibility
def get_color(accessibility):
    if accessibility == 'good':
        return 'darkgreen'
    elif accessibility == 'moderate':
        return 'orange'
    elif accessibility == 'poor':
        return 'red'
    return 'blue'  # Default if no accessibility information is available

# Add markers to the appropriate layer
for _, row in all_markers.iterrows():
    tooltip = row['name']
    
    # Popup content
    popup_content = f"<strong>{row['name']}</strong><br>"
    if pd.notna(row['transport']):
        popup_content += f"Public transport nearby: {'Yes' if row['transport'] else 'No'}<br>"
    if pd.notna(row['picture']):  # Check if image path is provided
        popup_content += f"<img src='{row['picture']}' width='150px'>"
    
    popup = folium.Popup(popup_content, max_width=300)
    
    # Determine icon and marker color based on type and accessibility
    icon_type = None
    if row['marker_type'] == 'resting_point':
        icon_type = folium.Icon(icon='umbrella-beach', prefix='fa', color=get_color(row['accessibility']))
    else:  # launch points
        icon_type = folium.Icon(icon='water', prefix='fa', color=get_color(row['accessibility']))
    
    # Create the marker
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup,
        tooltip=tooltip,
        icon=icon_type
    )
    
    # Add marker to the appropriate layer
    if row['marker_type'] == 'launch_with_transport':
        marker.add_to(launch_with_transport_layer)
    elif row['marker_type'] == 'launch_without_transport':
        marker.add_to(launch_without_transport_layer)
    else:
        marker.add_to(resting_point_layer)

# Add the layers to the map
launch_with_transport_layer.add_to(my_map)
launch_without_transport_layer.add_to(my_map)
resting_point_layer.add_to(my_map)

# Load GeoJSON line features
with open('berlin/routes.geojson') as f:
    geojson_data = json.load(f)

# Line features layer
line_layer = folium.FeatureGroup(name='Line Features', show=True)

# Function to calculate line length
def calculate_length(coords):
    total_length = 0
    for i in range(len(coords) - 1):
        start_point = (coords[i][1], coords[i][0])  # (lat, lon)
        end_point = (coords[i+1][1], coords[i+1][0])
        total_length += geopy.distance.distance(start_point, end_point).km
    return total_length

# Add GeoJSON lines with tooltips and popups
for feature in geojson_data['features']:
    coords = feature['geometry']['coordinates']
    start_location = feature['properties']['start']
    end_location = feature['properties']['end']
    
    # Calculate line length
    line_length = calculate_length(coords)
    
    # Create a GeoJson object for the line
    geojson = folium.GeoJson(
        data=feature,
        tooltip=f"Length: {line_length:.2f} km",
        popup=folium.Popup(
            f"<strong>Start:</strong> {start_location}<br><strong>End:</strong> {end_location}<br><strong>Length:</strong> {line_length:.2f} km",
            max_width=300
        )
    )
    
    # Add the GeoJson line to the line layer
    geojson.add_to(line_layer)

# Add the line layer to the map
line_layer.add_to(my_map)

# Add a layer control panel to toggle between the layers
folium.LayerControl().add_to(my_map)

# Add device location control to the map
LocateControl(auto_start=False).add_to(my_map)

# Add a draw tool to allow users to add temporary markers
Draw(export=True).add_to(my_map)

# Save the map to an HTML file
my_map.save('berlin/map.html')
